In [1]:
import pandas as pd
import mapply
import requests
from bs4 import BeautifulSoup

mapply.init(
    n_workers=32,
    chunk_size=50,
    max_chunks_per_worker=0,
    progressbar=False
)

In [2]:
OUTPUT_FILE = 'rights.csv.gz'

In [3]:
def download_html(url):
    try:
        r = requests.get(url, timeout=10)
        if r.status_code == 200:
            return r.content
        else:
            print('ERROR:', r.status_code)
    except Exception as e:
        print('ERROR', e)
    return ''

In [4]:
from collections import OrderedDict

meta_cols = OrderedDict({'name_of_ryot': 'lblName',
 'name_of_father': 'lblFHName',
 'residence': 'lblAddress',
 'jati': 'lblCaste',
 'thana': 'lblRTNO',
 'district': 'LblDis',
 'anchal': 'LblCir',
 'village': 'LblMau',
 'khatadari': 'LblLanHolId',
 'khata': 'LblKhataNo',
 'name_of_ryot2': 'lblOther',
})
cols = ['khasra', 'khet_boundary', 'land_type', 'area_acre', 'area_decimal',
        'area_hectare', 'dhakal_swaroop', 'lagaan', 'terms', 'gair_dakhildar',
        'orders', 'jamabandi']

def parse_html(url, html):
    meta = {'url': url}
    soup = BeautifulSoup(html)
    for col in meta_cols:
        span = soup.find('span', {'id': meta_cols[col]})
        if span:
            text = span.text.strip()
        else:
            text = ''
        #print(col, text)
        meta[col] = text
    table = soup.find('table', {'class': 'indexTable1'})
    rows = []
    if table:
        for tr in table.find_all('tr'):
            row = meta.copy()
            skip = False
            for i, td in enumerate(tr.find_all('td')):
                if 'colspan' in td.attrs:
                    skip = True
                    break
                row[cols[i]] =  td.text.strip()
            if skip:
                continue
            rows.append(row)
    else:
        rows.append(meta.copy())
    return rows

In [5]:
def scrape_rights(r):
    html = download_html(r.Record_of_Rights)
    rows = parse_html(r.Record_of_Rights, html)
    return pd.DataFrame(rows)

In [6]:
%%time
first = True
for i, df in enumerate(pd.read_csv('account_unique.csv.gz', chunksize=10000)):
    print(i)
    rdf = df.mapply(lambda r: scrape_rights(r), axis=1)
    for adf in rdf:
        if first:
            adf.to_csv(OUTPUT_FILE, index=False, header=first, compression='gzip')
            first = False
        else:
            adf.to_csv(OUTPUT_FILE, mode='a', index=False, header=first, compression='gzip')
    if i >= 9:
        break


Using more workers (32) than is sensible (3). For CPU-bound operations, consider lowering n_workers to avoid bottlenecks on the physical CPUs


0


Using more workers (32) than is sensible (3). For CPU-bound operations, consider lowering n_workers to avoid bottlenecks on the physical CPUs


1


Using more workers (32) than is sensible (3). For CPU-bound operations, consider lowering n_workers to avoid bottlenecks on the physical CPUs


...

7


Using more workers (32) than is sensible (3). For CPU-bound operations, consider lowering n_workers to avoid bottlenecks on the physical CPUs


8


Using more workers (32) than is sensible (3). For CPU-bound operations, consider lowering n_workers to avoid bottlenecks on the physical CPUs


9
CPU times: user 1min 45s, sys: 7.16 s, total: 1min 53s
Wall time: 51min 37s


In [9]:
%%time
first = False
for i, df in enumerate(pd.read_csv('account_unique.csv.gz', chunksize=10000, skiprows=range(1, 10*10000))):
    print(i)
    rdf = df.mapply(lambda r: scrape_rights(r), axis=1)
    for adf in rdf:
        if first:
            adf.to_csv(OUTPUT_FILE, index=False, header=first, compression='gzip')
            first = False
        else:
            adf.to_csv(OUTPUT_FILE, mode='a', index=False, header=first, compression='gzip')
    if i >= 49:
        break

Using more workers (32) than is sensible (3). For CPU-bound operations, consider lowering n_workers to avoid bottlenecks on the physical CPUs


0


Using more workers (32) than is sensible (3). For CPU-bound operations, consider lowering n_workers to avoid bottlenecks on the physical CPUs


1


Using more workers (32) than is sensible (3). For CPU-bound operations, consider lowering n_workers to avoid bottlenecks on the physical CPUs


...

47


Using more workers (32) than is sensible (3). For CPU-bound operations, consider lowering n_workers to avoid bottlenecks on the physical CPUs


48


Using more workers (32) than is sensible (3). For CPU-bound operations, consider lowering n_workers to avoid bottlenecks on the physical CPUs


49
CPU times: user 8min 47s, sys: 51 s, total: 9min 38s
Wall time: 4h 11min 35s


In [10]:
%%time
first = False
for i, df in enumerate(pd.read_csv('account_unique.csv.gz', chunksize=10000, skiprows=range(1, 60*10000))):
    print(i)
    rdf = df.mapply(lambda r: scrape_rights(r), axis=1)
    for adf in rdf:
        if first:
            adf.to_csv(OUTPUT_FILE, index=False, header=first, compression='gzip')
            first = False
        else:
            adf.to_csv(OUTPUT_FILE, mode='a', index=False, header=first, compression='gzip')
    if i >= 49:
        break

Using more workers (32) than is sensible (3). For CPU-bound operations, consider lowering n_workers to avoid bottlenecks on the physical CPUs


0


Using more workers (32) than is sensible (3). For CPU-bound operations, consider lowering n_workers to avoid bottlenecks on the physical CPUs


1


Using more workers (32) than is sensible (3). For CPU-bound operations, consider lowering n_workers to avoid bottlenecks on the physical CPUs


...

47


Using more workers (32) than is sensible (3). For CPU-bound operations, consider lowering n_workers to avoid bottlenecks on the physical CPUs


48


Using more workers (32) than is sensible (3). For CPU-bound operations, consider lowering n_workers to avoid bottlenecks on the physical CPUs


49
CPU times: user 9min 34s, sys: 56.9 s, total: 10min 31s
Wall time: 4h 40min 27s


In [6]:
%%time
first = False
for i, df in enumerate(pd.read_csv('account_unique.csv.gz', chunksize=10000, skiprows=range(1, 110*10000))):
    print(i)
    rdf = df.mapply(lambda r: scrape_rights(r), axis=1)
    for adf in rdf:
        if first:
            adf.to_csv(OUTPUT_FILE, index=False, header=first, compression='gzip')
            first = False
        else:
            adf.to_csv(OUTPUT_FILE, mode='a', index=False, header=first, compression='gzip')
    if i >= 39:
        break

Using more workers (20) than is sensible (3). For CPU-bound operations, consider lowering n_workers to avoid bottlenecks on the physical CPUs


0


Using more workers (20) than is sensible (3). For CPU-bound operations, consider lowering n_workers to avoid bottlenecks on the physical CPUs


1
ERROR HTTPConnectionPool(host='land.bihar.gov.in', port=80): Read timed out. (read timeout=10)
ERROR ERRORHTTPConnectionPool(host='land.bihar.gov.in', port=80): Read timed out. (read timeout=10) ERROR HTTPConnectionPool(host='land.bihar.gov.in', port=80): Read timed out. (read timeout=10)HTTPConnectionPool(host='land.bihar.gov.in', port=80): Read timed out. (read timeout=10)


ERROR HTTPConnectionPool(host='land.bihar.gov.in', port=80): Read timed out. (read timeout=10)
ERROR HTTPConnectionPool(host='land.bihar.gov.in', port=80): Read timed out. (read timeout=10)
ERROR HTTPConnectionPool(host='land.bihar.gov.in', port=80): Read timed out. (read timeout=10)
ERROR HTTPConnectionPool(host='land.bihar.gov.in', port=80): Read timed out. (read timeout=10)
ERROR HTTPConnectionPool(host='land.bihar.gov.in', port=80): Read timed out. (read timeout=10)
ERROR HTTPConnectionPool(host='land.bihar.gov.in', port=80): Read timed out. (read timeout=10)
ERROR HTTPConnectionPool(host='land.bihar.gov.in

Using more workers (20) than is sensible (3). For CPU-bound operations, consider lowering n_workers to avoid bottlenecks on the physical CPUs


...

37


Using more workers (20) than is sensible (3). For CPU-bound operations, consider lowering n_workers to avoid bottlenecks on the physical CPUs


38


Using more workers (20) than is sensible (3). For CPU-bound operations, consider lowering n_workers to avoid bottlenecks on the physical CPUs


39
CPU times: user 7min 6s, sys: 28.1 s, total: 7min 34s
Wall time: 4h 22min 32s


In [6]:
%%time
first = False
for i, df in enumerate(pd.read_csv('account_unique.csv.gz', chunksize=10000, skiprows=range(1, 150*10000))):
    print(i)
    rdf = df.mapply(lambda r: scrape_rights(r), axis=1)
    for adf in rdf:
        if first:
            adf.to_csv(OUTPUT_FILE, index=False, header=first, compression='gzip')
            first = False
        else:
            adf.to_csv(OUTPUT_FILE, mode='a', index=False, header=first, compression='gzip')
    if i >= 49:
        break

Using more workers (32) than is sensible (3). For CPU-bound operations, consider lowering n_workers to avoid bottlenecks on the physical CPUs


0


Using more workers (32) than is sensible (3). For CPU-bound operations, consider lowering n_workers to avoid bottlenecks on the physical CPUs


1


Using more workers (32) than is sensible (3). For CPU-bound operations, consider lowering n_workers to avoid bottlenecks on the physical CPUs


...

47


Using more workers (32) than is sensible (3). For CPU-bound operations, consider lowering n_workers to avoid bottlenecks on the physical CPUs


48


Using more workers (32) than is sensible (3). For CPU-bound operations, consider lowering n_workers to avoid bottlenecks on the physical CPUs


49
CPU times: user 9min 21s, sys: 43.5 s, total: 10min 5s
Wall time: 4h 24min 48s


In [7]:
%%time
first = False
for i, df in enumerate(pd.read_csv('account_unique.csv.gz', chunksize=10000, skiprows=range(1, 200*10000))):
    print(i)
    rdf = df.mapply(lambda r: scrape_rights(r), axis=1)
    for adf in rdf:
        if first:
            adf.to_csv(OUTPUT_FILE, index=False, header=first, compression='gzip')
            first = False
        else:
            adf.to_csv(OUTPUT_FILE, mode='a', index=False, header=first, compression='gzip')
    if i >= 199:
        break

Using more workers (32) than is sensible (3). For CPU-bound operations, consider lowering n_workers to avoid bottlenecks on the physical CPUs


0


Using more workers (32) than is sensible (3). For CPU-bound operations, consider lowering n_workers to avoid bottlenecks on the physical CPUs


1


Using more workers (32) than is sensible (3). For CPU-bound operations, consider lowering n_workers to avoid bottlenecks on the physical CPUs


...

197


Using more workers (32) than is sensible (3). For CPU-bound operations, consider lowering n_workers to avoid bottlenecks on the physical CPUs


198


Using more workers (32) than is sensible (3). For CPU-bound operations, consider lowering n_workers to avoid bottlenecks on the physical CPUs


199
CPU times: user 42min 2s, sys: 3min 24s, total: 45min 26s
Wall time: 18h 36min 13s


In [8]:
%%time
first = False
for i, df in enumerate(pd.read_csv('account_unique.csv.gz', chunksize=10000, skiprows=range(1, 400*10000))):
    print(i)
    rdf = df.mapply(lambda r: scrape_rights(r), axis=1)
    for adf in rdf:
        if first:
            adf.to_csv(OUTPUT_FILE, index=False, header=first, compression='gzip')
            first = False
        else:
            adf.to_csv(OUTPUT_FILE, mode='a', index=False, header=first, compression='gzip')
    if i >= 199:
        break

Using more workers (32) than is sensible (3). For CPU-bound operations, consider lowering n_workers to avoid bottlenecks on the physical CPUs


0


Using more workers (32) than is sensible (3). For CPU-bound operations, consider lowering n_workers to avoid bottlenecks on the physical CPUs


1


Using more workers (32) than is sensible (3). For CPU-bound operations, consider lowering n_workers to avoid bottlenecks on the physical CPUs


...

197


Using more workers (32) than is sensible (3). For CPU-bound operations, consider lowering n_workers to avoid bottlenecks on the physical CPUs


198


Using more workers (32) than is sensible (3). For CPU-bound operations, consider lowering n_workers to avoid bottlenecks on the physical CPUs


199
ERROR HTTPConnectionPool(host='land.bihar.gov.in', port=80): Read timed out. (read timeout=10)
CPU times: user 38min 58s, sys: 3min 30s, total: 42min 28s
Wall time: 17h 52min


In [9]:
%%time
first = False
for i, df in enumerate(pd.read_csv('account_unique.csv.gz', chunksize=10000, skiprows=range(1, 600*10000))):
    print(i)
    rdf = df.mapply(lambda r: scrape_rights(r), axis=1)
    for adf in rdf:
        if first:
            adf.to_csv(OUTPUT_FILE, index=False, header=first, compression='gzip')
            first = False
        else:
            adf.to_csv(OUTPUT_FILE, mode='a', index=False, header=first, compression='gzip')
    if i >= 199:
        break

Using more workers (32) than is sensible (3). For CPU-bound operations, consider lowering n_workers to avoid bottlenecks on the physical CPUs


0


Using more workers (32) than is sensible (3). For CPU-bound operations, consider lowering n_workers to avoid bottlenecks on the physical CPUs


1


Using more workers (32) than is sensible (3). For CPU-bound operations, consider lowering n_workers to avoid bottlenecks on the physical CPUs


...

Using more workers (32) than is sensible (3). For CPU-bound operations, consider lowering n_workers to avoid bottlenecks on the physical CPUs


184


Using more workers (32) than is sensible (3). For CPU-bound operations, consider lowering n_workers to avoid bottlenecks on the physical CPUs


185


AttributeError: 'int' object has no attribute 'to_csv'